In [7]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM,Bidirectional,Conv1D,MaxPool1D,Dense,Input, Flatten,Dropout,SpatialDropout1D,GlobalMaxPool1D
from keras.models import Sequential
from keras import backend as K
import keras
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df = pd.read_csv('train.csv', index_col=0)
X = df.comment_text
Y = df[['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']]
train_X, test_X, train_Y, test_Y = train_test_split(X,Y, test_size = 0.2,random_state = 10)

In [10]:
# non neural classification model
# LR
tok = nltk.tokenize.TweetTokenizer()
vectorizer = CountVectorizer(lowercase= True, stop_words='english', ngram_range=(1,2),tokenizer=tok.tokenize)
train_X_vec = vectorizer.fit_transform(train_X)
test_X_vect = vectorizer.transform(test_X)

In [11]:
#LR
lr_tox = LogisticRegression()
lr_stox = LogisticRegression()
lr_ob = LogisticRegression()
lr_th = LogisticRegression()
lr_in = LogisticRegression()
lr_id = LogisticRegression()

In [14]:
lr_tox.fit(train_X_vec,train_Y['toxic'])
lr_stox.fit(train_X_vec, train_Y['severe_toxic'])
lr_ob.fit(train_X_vec, train_Y['obscene'])
lr_th.fit(train_X_vec, train_Y['threat'])
lr_in.fit(train_X_vec, train_Y['insult'])
lr_id.fit(train_X_vec, train_Y['identity_hate'])

/Users/tianyuyang/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/tianyuyang/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
tox_pred = lr_tox.predict_proba(test_X_vect)[:,1]
stox_pred = lr_stox.predict_proba(test_X_vect)[:,1]
ob_pred =lr_ob.predict_proba(test_X_vect)[:,1]
th_pred = lr_th.predict_proba(test_X_vect)[:,1]
in_pred = lr_in.predict_proba(test_X_vect)[:,1]
id_pred = lr_id.predict_proba(test_X_vect)[:,1]

In [20]:
tox_auc = roc_auc_score(test_Y['toxic'], tox_pred)
stox_auc =roc_auc_score(test_Y['severe_toxic'], stox_pred)
ob_auc = roc_auc_score(test_Y['obscene'], ob_pred)
th_auc = roc_auc_score(test_Y['threat'], th_pred)
in_auc = roc_auc_score(test_Y['insult'], in_pred)
id_auc = roc_auc_score(test_Y['identity_hate'], id_pred)

In [23]:
pd.Series([tox_auc,stox_auc,ob_auc ,th_auc ,in_auc,id_auc], test_Y.columns).to_csv('lr_result.csv')